In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
ANALYTICS_BASE_URL = "https://api.info.v2.tibetswap.io/"

In [3]:
base_url = ANALYTICS_BASE_URL

In [4]:
def make_url(base, verb, params=None):
    """Generate a URL to query a REST API"""

    param_string = ''
    if params is not None:
        param_string = '?' + '&'.join([f'{k}={v}' for k, v in params.items()])
    return base + verb + param_string

In [5]:
class Pairs:
    """
    Handle data about the available asset pairs.
    
    Assumes each token is paired with XCH.
    """
    
    def __init__(self, base_url=ANALYTICS_BASE_URL):
        self.base_url = base_url
        self.data = None
        
    def populate(self):
        url = make_url(self.base_url, 'pairs')
        response = requests.get(url)
        response.raise_for_status()
        self.data = pd.json_normalize(response.json())
        
    def refresh(self):
        self.populate()
        
    def get_data_item(self, entity='short_name', field='launcher_id', name='STDG'):
        if self.data is None:
            self.populate()
        return self.data[self.data[entity] == name][field].values[0]
    
    def get_launcher_id(self, token):
        return self.get_data_item(field='launcher_id', name=token)
    
    def get_xch_reserve(self, token):
        return self.get_data_item(field='xch_reserve', name=token).astype('float') / 1e12
    
    def get_transactions(self, token, lookback=7, op=None, limit=None):
        params = {}
        params['pair_launcher_id'] = self.get_launcher_id(token)
        if op is not None:
            params['operation'] = op
        if limit is not None:
            params['limit'] = limit
        url = make_url(base_url, 'transactions', params)
        response = requests.get(url)
        response.raise_for_status()
        df = pd.json_normalize(response.json())
        df['time'] = pd.to_datetime(df.timestamp, unit='s')
        start = datetime.now() - timedelta(days=lookback)
        return df[df.time > start].copy()
    
    def get_volume(self, token, lookback=7):
        df = self.get_transactions(token, lookback=lookback, op='SWAP')
        return df['state_change.xch'].abs().sum().astype('float') / 1e12

In [6]:
class Pair:
    """
    Get information about a particular asset pair.
    
    Assumes a token is paired with XCH.
    """
    
    def __init__(self, pairs, token_short_name):
        self.pairs = pairs
        self.token = token_short_name
        self.launcher_id = pairs.get_launcher_id(self.token)
        
    def refresh(self):
        self.pairs.refresh()
        # Will also need to refresh any token-specific caches, but there aren't any yet
        
    def get_transactions(self, lookback=7, op=None, limit=None):
        return self.pairs.get_transactions(self.token, lookback=7, op=op, limit=limit)
        
    def get_volume(self, lookback=7):
        return self.pairs.get_volume(self.token, lookback)
    
    def get_xch_reserve(self):
        return self.pairs.get_xch_reserve(self.token)

    def get_periodic_yield(self, lookback=7):
        return self.get_volume(lookback) * 0.0035 / self.get_xch_reserve()
    
    def get_annualized_yield(self, lookback=7):
        return (1 + self.get_periodic_yield(lookback)) ** (365.25 / lookback) - 1
    
    def get_price_impact_threshold(self, impact=0.02):
        return impact * self.get_xch_reserve() / 2
    
    def get_recovery_period(self, size=1, unit=7):
        return (2 * size / self.get_xch_reserve()) / self.get_periodic_yield(lookback=unit)

In [7]:
pairs = Pairs()
stdg = Pair(pairs, 'STDG')
stdg.get_volume()

1.597176087041

In [8]:
stdg.get_xch_reserve()

4.574957305951

In [9]:
stdg.get_annualized_yield()

0.06579159215779229

In [10]:
stdg.get_price_impact_threshold()

0.04574957305951

In [11]:
stdg.get_recovery_period()

357.7743093356886

In [12]:
usds = Pair(pairs, 'USDS')
usds.get_volume()

48.404130082718

In [13]:
usds.get_recovery_period()

11.805368063676696

In [14]:
incl4 = Pair(pairs, 'INCL4')
incl4.get_recovery_period()

80.25438164575237

In [15]:
incl4.get_recovery_period(0.1)

8.025438164575238

In [16]:
incl4.get_annualized_yield(), incl4.get_xch_reserve()

(0.3122653347669977, 4.772485328489)

In [17]:
incl4.get_volume()

7.120216488003

In [18]:
stdg.get_transactions(op='SWAP')

,pair_launcher_id,operation,height,coin_id,timestamp,state_change.xch,state_change.token,state_change.liquidity,time
0,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783208,bd981a80d3968688bb0800373b0afbd88fc910f6f2494e...,1686365673,49999946955,-452990,0,2023-06-10 02:54:33
1,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783114,228ef267b62baefbcd69af238c25350b1904f9495112b5...,1686363923,19999976134,-184183,0,2023-06-10 02:25:23
2,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3783017,02a71eb068f6076f435adf5d6c2ecfa788596ba0b1b4c9...,1686362585,499999986537,-5247188,0,2023-06-10 02:03:05
3,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3782598,20466e8d2cfecc7285ddc2b92bf3f35a374a942fafaf41...,1686354623,-466867310522,5000000,0,2023-06-09 23:50:23
4,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3775212,f409cd53c3f28acd260a5553ce0da2954c46e0b91adbc3...,1686216158,52612677973,-500000,0,2023-06-08 09:22:38
5,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3771037,da48e215f3b008bf5da4f30272da1a1dbcd0b660aa6002...,1686136123,200000000000,-2025017,0,2023-06-07 11:08:43
6,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3767064,3e9b740737c6a183dc54e2573e097dc3d210c072dfc4a2...,1686062718,-154143842754,1600000,0,2023-06-06 14:45:18
7,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3766871,4c6a176a3ba3c5a8296e182c11941b04928142a8d7c041...,1686059463,80000000000,-803282,0,2023-06-06 13:51:03
8,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3765260,f0d019a761686c57d803d3c1b7b07a46f8c80840ce47e8...,1686029849,-73552346166,750000,0,2023-06-06 05:37:29


In [19]:
pairs.data.head()

,launcher_id,name,short_name,image_url,asset_id,current_coin_id,xch_reserve,token_reserve,liquidity,trade_volume
0,b7772fbcb3a297555f2dd75368ee85069c50d294902b40...,Stably USD,USDS,https://nftstorage.link/ipfs/bafybeibsolxdn3yn...,6d95dae356e32a71db5ddcb42224754a02524c615c5fc3...,df1398f0259c12688c09a3c9953caea0a76894b84166e7...,343424796224097,11365179,11947880,355449018579249
1,636a5be72b9974d28393fbff1249e28dcdfb86b0bb9245...,Big Brain Token,BBT,https://bigbrain.at/bigbraintoken.png,9e04d998a90bca05bdbc301548c83104d79256f1b00bb2...,c27842232b19c76136d449608a918ec8ac0f88f4befdfa...,182487000625949,211680530617,6112833045903,299407566346984
2,b62577aed2b09182f3db8b21049d687b597f1526fb8826...,PepeCoin,PEPE,https://nftstorage.link/ipfs/bafybeibtolqdipv4...,ec9d874e152e888231024c72e391fc484e8b6a1cf74443...,099c82379167521aef03678d248449ca5933233ee42423...,132291766554397,46333942679,38688759926,350571398964935
3,6dc7a439046ae61bf51e3af7e960ab691dbe6d58ddb289...,DeltaXtrade,DXT,https://nftstorage.link/ipfs/bafybeihklkvffcwg...,a7c8135b134e1a477563480febefd2e2091afe15d24ab5...,5b88ea0d81fe7c349232c5180d47aad35418e3c7b60a1d...,128235801668250,137577,158578,9373639859586
4,c0952d9c3761302a5545f4c04615a855396b1570be5c61...,Dexie Bucks,DBX,https://nftstorage.link/ipfs/bafybeidiupubzjzn...,db1a9020d48d9d4ad22631b66ab4b9ebd3637ef7758ad3...,58bbc3611558a7f7f8ec9e0326d080edba013325c1ecc7...,112536160469925,36210179,31594406,95269462200480
